In [1]:

import talib as ta
import tushare as ts
import numpy as np
import pandas as pd
import time
import sys
import datetime
import json
import ast
import pymongo
import importlib
from pandas.io.json import json_normalize
sys.path.append(r"D:\Jupyter_Pj\pyModule")
import mongodbOp
importlib.reload(mongodbOp)

<module 'mongodbOp' from 'D:\\Jupyter_Pj\\pyModule\\mongodbOp.py'>

In [2]:
db_name = "stock"
daily_col_name = "stock_daily"
stock_basic_col_name = "stock_basic_info"
stock_divid_factors_col_name = "stock_divid_factors"
stock_db_info_name = "stock_db_info"

daily_start_date = "20120801"
daily_end_date   = "20220811"

In [3]:
db_op = mongodbOp.MongoDbOp()
db_op.connect_To_MongoDb(db_name= db_name)

In [4]:
def get_limit_up_date(dateframe,return_Peroid =10, x_day=20, y_day= 40, z_day= 60):
    
    zt = dateframe[dateframe["pct_chg"] >9.9]
    zt = zt.dropna()
    
    val_return = pd.DataFrame(columns=['code','zhangtingri','close','zhuangzheri','zzr_close','n_daytupo','tupo_close','x_dayprice','x_dayhigh','y_dayprice','y_dayhigh','z_dayprice','z_dayhigh'])
    for j in zt.index:
        val_return.loc[j,'code'] = zt.loc[j,'ts_code']
        val_return.loc[j,'zhangtingri'] = zt.loc[j,'trade_date']
        val_return.loc[j,'close'] = zt.loc[j,'close']
    
    val_return = val_return.sort_index()
    
    
    for m in zt.index:
        
        ztt = dateframe.loc[m:,["trade_date","close"]]
        
        j = m
        k = 0
        for j in ztt.index:           # 涨停后， 找到转跌的前一天，以这天开始取 review_period_1 天的数据
            if( (k+1) >= len(ztt) ):
                break # 超过 ztt 的范围，退出查找
            if(ztt.iloc[k+1]['close'] < ztt.iloc[k]['close']):
                val_return.loc[m,'zzr_close'] = dateframe.loc[j,'close']
                val_return.loc[m,'zhuangzheri'] = k
                break
            k +=1
       #     if( k > return_Peroid):
       #         val_return.loc[i,'ztr_close'] = 0
      #          val_return.loc[i,'zhuangzheri'] = 999
      #          continue
        
        high_up_index = j
        ztd  =  dateframe.loc[j:].iloc[0:z_day+1]
    
        ztv = ztd.iloc[0:return_Peroid]
        
        # 查找突破日， 把突破日和当天的收盘价保存下来
        k =0
        for j in ztv.index:
            k +=1
            if(ztv.loc[j,'close'] > ztv.loc[high_up_index,'close']):
                val_return.loc[m,'tupo_close'] = ztv.loc[j,'close']
                val_return.loc[m,'n_daytupo'] = k
                break
        
        # fillout the data
        if( len(ztd)> x_day):         
            val_return.loc[m,'x_dayprice'] = ztd.iloc[x_day]['close']
        else:
            val_return.loc[m,'x_dayprice'] = ztd.iloc[-1]['close']
        val_return.loc[m,'x_dayhigh'] = ztd.loc[:,'close'][0:x_day].max()

        if( len(ztd)> y_day):   
            val_return.loc[m,'y_dayprice'] = ztd.iloc[y_day]['close']
        else:
            val_return.loc[m,'y_dayprice'] = ztd.iloc[-1]['close']
        val_return.loc[m,'y_dayhigh'] = ztd.loc[:,'close'][0:y_day].max()

        if( len(ztd)> z_day): 
            val_return.loc[m,'z_dayprice'] = ztd.iloc[z_day]['close']
        else:
            val_return.loc[m,'z_dayprice'] = ztd.iloc[-1]['close']
        val_return.loc[m,'z_dayhigh'] = ztd.loc[:,'close'][0:z_day].max()
    
    
    return (val_return.reset_index(drop=True) )
    

In [12]:
pro = ts.pro_api('af5f423bb57f376e71c2903058362f8e3e053fe30f486463f9bdac8e')
#sb=pd.read_csv('stockInfo\StockBasic.csv')
sb = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,market,list_date')

file_name = "stockInfo\\"+sb.loc[1,'ts_code'] +".csv"
sb.to_csv(file_name,index = False, encoding='utf_8_sig')

In [14]:
sb

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,ST国华,深圳,软件服务,主板,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
4859,871981.BJ,871981,晶赛科技,None,None,北交所,20211115
4860,872925.BJ,872925,锦好医疗,None,None,北交所,20211025
4861,873169.BJ,873169,七丰精工,None,None,北交所,20220415
4862,873223.BJ,873223,荣亿精密,None,None,北交所,20220609


In [19]:
result_all = pd.DataFrame(data = None, columns=['code','zhangtingri','close','zhuangzheri','zzr_close','n_daytupo','tupo_close','x_dayprice','x_dayhigh','y_dayprice','y_dayhigh','z_dayprice','z_dayhigh'])

for i in range(101,105):
    #df = ts.pro_bar(ts_code=sb.loc[i,'ts_code'], adj='qfq', start_date='20181229', end_date='20220729')
    df = db_op.find_any(daily_col_name, {"stock_code": sb.loc[i,'ts_code'],"trade_date":{'$gt':"20200101"}})
    
   # df.rename(columns={"stock_code":"ts_code"})
    df.rename(columns = {"stock_code":"ts_code"},inplace=True)
    #file_name = "stockInfo\\"+sb.loc[i,'ts_code'] +".csv" 
    #df.to_csv(file_name,index = False, encoding='utf_8_sig')
    
    val_ret = get_limit_up_date(df)
    result_all = pd.concat([result_all,val_ret],ignore_index=True)
    

D:\Jupyter_Pj\pyModule\mongodbOp.py:141: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(ret)
D:\Jupyter_Pj\pyModule\mongodbOp.py:141: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(ret)
D:\Jupyter_Pj\pyModule\mongodbOp.py:141: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(ret)
D:\Jupyter_Pj\pyModule\mongodbOp.py:141: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(ret)


In [20]:

#file_name = "stockInfo\\"+"analyResult" +".csv" 
#result_all.to_csv(file_name,index = False, encoding='utf_8_sig')

result_all

,code,zhangtingri,close,zhuangzheri,zzr_close,n_daytupo,tupo_close,x_dayprice,x_dayhigh,y_dayprice,y_dayhigh,z_dayprice,z_dayhigh
0,000506.SZ,20220810,3.66,0,3.66,NaN,NaN,3.08,3.66,3.32,3.66,3.07,3.66
1,000506.SZ,20220510,3.5,0,3.5,NaN,NaN,2.9,3.5,3.52,3.87,4.03,4.12
2,000506.SZ,20220506,2.97,0,2.97,NaN,NaN,2.89,2.99,3.61,3.87,3.83,4.12
3,000506.SZ,20220505,2.7,0,2.7,9,2.72,2.88,2.99,3.73,3.87,3.73,4.12
4,000506.SZ,20220309,3.87,0,3.87,9,3.96,3.99,4.12,3.5,4.25,2.98,4.25
5,000506.SZ,20220104,3.53,0,3.53,NaN,NaN,3.06,3.53,3.13,3.53,3.52,3.53
6,000506.SZ,20211210,3.31,0,3.31,NaN,NaN,3.15,3.31,3.18,3.31,3.52,3.9
7,000506.SZ,20210928,3.36,0,3.36,4,3.42,3.75,3.9,3.66,3.9,2.08,4.49
8,000506.SZ,20210823,3.31,0,3.31,6,3.41,4.08,4.49,2.1,4.49,2.22,4.49
9,000506.SZ,20210727,4.49,0,4.49,NaN,NaN,2.07,4.49,2.2,4.49,2.07,4.49
